This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

# Section 1 Generate a list of house listings.

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "voc-12993709331266773773364678ee3ae65f628.88770020"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm_generate_listings = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 3000)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
,  warnings.warn(
,/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
,  warnings.warn(


In [2]:
example_listing = """Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
"Green Oaks","$800,000",3,2,"2,000 sqft","Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.","Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
"""

In [3]:
# Generate real estate listings 
instructions = f"""
Generate another 10 different realistic real estate listings with some different locations, house types and prices. New listings should be added after this example: 

{example_listing}
"""
generated_listings = llm_generate_listings.predict(instructions).replace("\n\n", "\n")

In [4]:
print(generated_listings)

"Seaside Retreat","$1,200,000",4,3,"2,500 sqft","Escape to this stunning seaside retreat located just steps away from the beach. This 4-bedroom, 3-bathroom home features panoramic ocean views from every room, a gourmet kitchen with high-end appliances, and a spacious deck perfect for entertaining. Enjoy the sound of crashing waves and salty sea breeze in this coastal paradise.","Seaside Retreat is a sought-after beachfront community known for its pristine sandy beaches, charming seaside shops, and delicious seafood restaurants. Residents can enjoy surfing, paddleboarding, and beach bonfires right in their backyard."
,"Mountain Hideaway","$900,000",3,2.5,"2,300 sqft","Discover this hidden mountain hideaway nestled in the tranquil forests of the Sierra Nevada. This 3-bedroom, 2.5-bathroom home features vaulted ceilings, a cozy stone fireplace, and a wrap-around deck with breathtaking mountain views. Escape the hustle and bustle of city life and embrace the serenity of nature in this peac

In [5]:
file_name = "generated_house_listings.csv"

# Open the file in write mode and save the string
with open(file_name, "w") as file:
    file.write(example_listing+generated_listings)

# Section 2 Vector DB setup and store data

In [6]:
# store the embeddings of listings in lance db 

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = CSVLoader(file_path='generated_house_listings.csv')
house_listings = loader.load()
print(house_listings)

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(house_listings, embeddings)


[Document(page_content='Neighborhood: Green Oaks\nPrice: $800,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,000 sqft\nDescription: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.\nNeighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

# Section 3 Semantic Search

In [35]:
# Collect User preference, fake questions and answers
questions = [   
                "How big do you want your house to be?" ,
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [36]:
answers_in_string = " ".join(answers)
preferred_listings = db.similarity_search(answers_in_string, k=3)
print(preferred_listings)

[Document(page_content='Neighborhood: Green Oaks\nPrice: $800,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,000 sqft\nDescription: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.\nNeighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

# Section 4 Search and augment listing based on results

In [37]:
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user a property listing based on their answers to personal questions. Ask user {len(questions)} questions""")
for i in range(len(questions)):
    history.add_ai_message(questions[i])
    history.add_user_message(answers[i])



In [38]:
from typing import Any, Dict, Optional, Tuple

llm_agent = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 3000)

class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
    
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)


In [39]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain

RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Real Estate Recommender. 
The AI collects user preference and provides real estate for a human based on the house listing. 

Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
# create a recommendation conversation chain that will let us ask AI for recommendations on all movies
recommender = ConversationChain(llm=llm_agent, verbose=True, memory=conversational_memory, prompt=PROMPT)

In [40]:
preferred_listings[0].page_content.split("\n")

['Neighborhood: Green Oaks',
 'Price: $800,000',
 'Bedrooms: 3',
 'Bathrooms: 2',
 'House Size: 2,000 sqft',
 'Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.',
 'Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.']

In [41]:
def transform_csv_to_natural_language(listing):
    content = listing.page_content.split("\n")
    natural_listing = ""
    for line in content:
        natural_listing += (line + "\n")
    return natural_listing
    
query = "HERE ARE THE TOP 3 LISTINGS BASED ON USER PREFERENCE, SEPARATED BY EMPTY LINE: \n"
for listing in preferred_listings:
    query += (transform_csv_to_natural_language(listing)+"\n")

query += """

IN THE END, GENERATE A PERSONALIZED DESCRIPTIONS FOR THE MOST SUITABLE REAL ESTATE LISTING BASED ON BUYER PREFERENCES. THE DESCRIPTIONS SHOULD BE UNIQUE, APPEALING, AND TAILORED TO THE PREFERENCES PROVIDED WITHOUT CHANGING FACTUAL INFORMATION.
"""

In [42]:
print(query)

HERE ARE THE TOP 3 LISTINGS BASED ON USER PREFERENCE, SEPARATED BY EMPTY LINE: 
,Neighborhood: Green Oaks
,Price: $800,000
,Bedrooms: 3
,Bathrooms: 2
,House Size: 2,000 sqft
,Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
,Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to pub

In [43]:
prediction = recommender.predict(input=query)
print(prediction)


,
,> Entering new ConversationChain chain...
,Prompt after formatting:
,The following is a friendly conversation between a human and an AI Real Estate Recommender. 
,The AI collects user preference and provides real estate for a human based on the house listing. 
,
,Personal Questions and Answers:
,Human: You are AI that will recommend user a property listing based on their answers to personal questions. Ask user 5 questions
,AI: How big do you want your house to be?
,Human: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
,AI: What are 3 most important things for you in choosing this property?
,Human: A quiet neighborhood, good local schools, and convenient shopping options.
,AI: Which amenities would you like?
,Human: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
,AI: Which transportation options are important to you?
,Human: Easy access to a reliable bus line, proximity to a major highway, and bike-friend